In [ ]:
import glob
import re
import json
from collections import Counter

## Read in and do basic cleaning

In [ ]:
all_data = []

In [ ]:
def process_folder(folder_path):
    all_text_files = glob.glob(folder_path)
    file_names = [full_path[(full_path.rfind('/') + 1):] for full_path in all_text_files]
    projects = list(set([file_name[:file_name.find("_")] for file_name in file_names]))
    return all_text_files, file_names, projects

In [ ]:
def clean_doc_text(doc_text):
    doc_text = re.sub("\u00b7", "", doc_text)
    doc_text = re.sub(" +", " ", doc_text)
    doc_text = re.sub("\.+", "\.", doc_text)
    return doc_text.strip()

In [ ]:
def process_and_append(all_text_files, document_type):
    for full_path in all_text_files:
        file_name = full_path[(full_path.rfind('/') + 1):]
        data_entry = { "project_id": file_name[:file_name.find("_")] }
        with open(full_path, "r", encoding="ISO-8859-1") as f:
            try:
                doc_text = f.read()
                data_entry["document_text"] = clean_doc_text(doc_text)
                data_entry["document_type"] = document_type
            except:
                print("Error on project: ", project_id, " file: ", file_name)
                raise

        all_data.append(data_entry)

In [ ]:
pad_text_files, file_names, pad_projects = process_folder("../data/text_data/pad_documents/*.txt")

In [ ]:
process_and_append(pad_text_files, "APPROVAL")

In [ ]:
pad_entries = [entry for entry in all_data if entry["document_type"] == "APPROVAL"]
pad_entries[0]

In [ ]:
len(pad_entries)

In [ ]:
icr_text_files, icr_file_names, icr_projects = process_folder("../data/text_data/icr_documents/*.txt")

In [ ]:
process_and_append(icr_text_files, "REVIEW")

In [ ]:
icr_entries = [entry for entry in all_data if entry["document_type"] == "REVIEW"]
icr_entries[0]

In [ ]:
len(all_data)

In [ ]:
file_project_ids = [entry["project_id"] for entry in all_data]

In [ ]:
counter = Counter(file_project_ids)
# proj_max = 
# print("")
# counter[proj_max]
counter.most_common()

## Write to consolidated file

In [ ]:
all_docs_file = "../data/text_data/wb_project_documents.jsonl"

In [ ]:
with open(all_docs_file, "w") as f:
    for entry in all_data:
        f.write(json.dumps(entry) + "\n")

In [ ]:
len(all_data)

In [ ]:
length = sum(1 for line in open(all_docs_file))
length

## Check loads via HF Datasets

In [ ]:
import datasets

In [ ]:
datasets.set_caching_enabled(False)

In [ ]:
ds = datasets.load_dataset("json", data_files="../data/text_data/wb_project_documents.jsonl")

In [ ]:
ds